https://auto.gluon.ai/0.1.0/tutorials/tabular_prediction/tabular-indepth.html
    
# Predicting Columns in a Table - In Depth

Tip: If you are new to AutoGluon, review "Predicting Columns in a Table - Quick Start" to learn the basics of the AutoGluon API.

https://auto.gluon.ai/0.1.0/tutorials/tabular_prediction/tabular-quickstart.html#sec-tabularquick

This tutorial describes how you can exert greater control when using AutoGluon’s `fit()` or `predict()`. Recall that to maximize predictive performance, you should always first try `fit()` with all default arguments except `eval_metric` and `presets`, before you experiment with other arguments covered in this in-depth tutorial like `hyperparameter_tune_kwargs`, `hyperparameters`, `num_stack_levels`, `num_bag_folds`, `num_bag_sets`, etc.

Using the same census data table as in the "Predicting Columns in a Table - Quick Start" tutorial, we’ll now predict the occupation of an individual - a multiclass classification problem. Start by importing AutoGluon’s `TabularPredictor` and `TabularDataset`, and loading the data.

In [1]:
from autogluon.tabular import TabularDataset, TabularPredictor
import numpy as np

In [2]:
train_data = TabularDataset('https://autogluon.s3.amazonaws.com/datasets/Inc/train.csv')
subsample_size = 500  # subsample subset of data for faster demo, try setting this to much larger values
train_data = train_data.sample(n=subsample_size, random_state=0)
print(train_data.head())

       age workclass  fnlwgt      education  education-num  \
6118    51   Private   39264   Some-college             10   
23204   58   Private   51662           10th              6   
29590   40   Private  326310   Some-college             10   
18116   37   Private  222450        HS-grad              9   
33964   62   Private  109190      Bachelors             13   

            marital-status        occupation    relationship    race      sex  \
6118    Married-civ-spouse   Exec-managerial            Wife   White   Female   
23204   Married-civ-spouse     Other-service            Wife   White   Female   
29590   Married-civ-spouse      Craft-repair         Husband   White     Male   
18116        Never-married             Sales   Not-in-family   White     Male   
33964   Married-civ-spouse   Exec-managerial         Husband   White     Male   

       capital-gain  capital-loss  hours-per-week  native-country   class  
6118              0             0              40   United-State

In [3]:
label = 'occupation'
print("Summary of occupation column: \n", train_data['occupation'].describe())

Summary of occupation column: 
 count                  500
unique                  15
top        Exec-managerial
freq                    77
Name: occupation, dtype: object


In [4]:
new_data = TabularDataset('https://autogluon.s3.amazonaws.com/datasets/Inc/test.csv')
test_data = new_data[5000:].copy()  # this should be separate data in your applications
y_test = test_data[label]
test_data_nolabel = test_data.drop(columns=[label])  # delete label column
val_data = new_data[:5000].copy()

metric = 'accuracy' # we specify eval-metric just for demo (unnecessary as it's the default)

## Specifying hyperparameters and tuning them

We first demonstrate hyperparameter-tuning and how you can provide your own validation dataset that AutoGluon internally relies on to: tune hyperparameters, early-stop iterative training, and construct model ensembles. One reason you may specify validation data is when future test data will stem from a different distribution than training data (and your specified validation data is more representative of the future data that will likely be encountered).

If you don’t have a strong reason to provide your own validation dataset, we recommend you omit the `tuning_data` argument. This lets AutoGluon automatically select validation data from your provided training set (it uses smart strategies such as stratified sampling). For greater control, you can specify the `holdout_fra`c argument to tell AutoGluon what fraction of the provided training data to hold out for validation.

**Caution:** Since AutoGluon tunes internal knobs based on this validation data, performance estimates reported on this data may be over-optimistic. For unbiased performance estimates, you should always call `predict()` on a separate dataset (that was never passed to `fit()`), as we did in the previous Quick-Start tutorial. We also emphasize that most options specified in this tutorial are chosen to minimize runtime for the purposes of demonstration and you should select more reasonable values in order to obtain high-quality models.

`fit()` trains neural networks and various types of tree ensembles by default. You can specify various hyperparameter values for each type of model. For each hyperparameter, you can either specify a single fixed value, or a search space of values to consider during hyperparameter optimization. Hyperparameters which you do not specify are left at default settings chosen automatically by AutoGluon, which may be fixed values or search spaces.

In [5]:
import autogluon.core as ag

In [6]:
# specifies non-default hyperparameter values for neural network models
#    num_epochs: number of training epochs (controls training time of NN models)
#    learning_rate: learning rate used in training (real-valued hyperparameter searched on log-scale)
#    activation: activation function used in NN (categorical hyperparameter, default = first entry)
#    layers: each choice for categorical hyperparameter 'layers' corresponds to list of sizes for each NN layer to use
#    dropout_prob: dropout probability (real-valued hyperparameter)
nn_options = {
    'num_epochs': 10,
    'learning_rate': ag.space.Real(1e-4, 1e-2, default=5e-4, log=True),
    'activation': ag.space.Categorical('relu', 'softrelu', 'tanh'),
    'layers': ag.space.Categorical([100], [1000], [200, 100], [300, 200, 100]),
    'dropout_prob': ag.space.Real(0.0, 0.5, default=0.1)
}

nn_options

{'num_epochs': 10,
 'learning_rate': Real: lower=0.0001, upper=0.01,
 'activation': Categorical['relu', 'softrelu', 'tanh'],
 'layers': Categorical[[100], [1000], [200, 100], [300, 200, 100]],
 'dropout_prob': Real: lower=0.0, upper=0.5}

In [7]:
# specifies non-default hyperparameter values for lightGBM gradient boosted trees
#    num_boost_round: number of boosting rounds (controls training time of GBM models)
#    num_leaves: number of leaves in trees (integer hyperparameter)
gbm_options = {  
    'num_boost_round': 100,
    'num_leaves': ag.space.Int(lower=26, upper=66, default=36)
}

gbm_options

{'num_boost_round': 100, 'num_leaves': Int: lower=26, upper=66}

In [8]:
# hyperparameters of each model type
# When these keys are missing from hyperparameters dict, no models of that type are trained
#    NN: NOTE: comment this line out if you get errors on Mac OSX
hyperparameters = {  
    'GBM': gbm_options,
    'NN': nn_options
}

hyperparameters

{'GBM': {'num_boost_round': 100, 'num_leaves': Int: lower=26, upper=66},
 'NN': {'num_epochs': 10,
  'learning_rate': Real: lower=0.0001, upper=0.01,
  'activation': Categorical['relu', 'softrelu', 'tanh'],
  'layers': Categorical[[100], [1000], [200, 100], [300, 200, 100]],
  'dropout_prob': Real: lower=0.0, upper=0.5}}

In [9]:
# train various models for ~2 min
# time_limit = 2*60

# https://www.kaggle.com/code/daikikatsuragawa/tps-mar-2021-benchmark-using-autogluon/notebook
# If the value of time_limit is too small, we will get the following error:
# ValueError: AutoGluon did not successfully train any models

time_limit = 10*60
time_limit

600

In [10]:
# try at most 5 different hyperparameter configurations for each type of model
num_trials = 5
num_trials

5

In [11]:
# to tune hyperparameters using Bayesian optimization routine with a local scheduler
search_strategy = 'auto'
search_strategy

'auto'

In [12]:
# HPO is not performed unless hyperparameter_tune_kwargs is specified
hyperparameter_tune_kwargs = {
    'num_trials': num_trials,
    'searcher': search_strategy
}

hyperparameter_tune_kwargs

{'num_trials': 5, 'searcher': 'auto'}

In [13]:
label

'occupation'

In [14]:
metric

'accuracy'

In [15]:
# NEED TO DEBUG
"""
predictor = TabularPredictor(label=label, eval_metric=metric).fit(
    train_data,
    tuning_data=val_data,
    time_limit=time_limit,
    hyperparameters=hyperparameters,
    hyperparameter_tune_kwargs=hyperparameter_tune_kwargs
)
"""

'\npredictor = TabularPredictor(label=label, eval_metric=metric).fit(\n    train_data,\n    tuning_data=val_data,\n    time_limit=time_limit,\n    hyperparameters=hyperparameters,\n    hyperparameter_tune_kwargs=hyperparameter_tune_kwargs\n)\n'

In [16]:
predictor = TabularPredictor(label=label, eval_metric=metric).fit(
    train_data,
    time_limit=time_limit
)

No path specified. Models will be saved in: "AutogluonModels/ag-20230517_165738/"
Beginning AutoGluon training ... Time limit = 600s
AutoGluon will save models to "AutogluonModels/ag-20230517_165738/"
AutoGluon Version:  0.7.0
Python Version:     3.9.11
Operating System:   Linux
Platform Machine:   x86_64
Platform Version:   #1 SMP Tue Mar 7 15:41:52 UTC 2023
Train Data Rows:    500
Train Data Columns: 14
Label Column: occupation
Preprocessing data ...
AutoGluon infers your prediction problem is: 'multiclass' (because dtype of label-column == object).
	First 10 (of 15) unique label values:  [' Exec-managerial', ' Other-service', ' Craft-repair', ' Sales', ' Prof-specialty', ' Protective-serv', ' ?', ' Adm-clerical', ' Machine-op-inspct', ' Tech-support']
	If 'multiclass' is not the correct problem_type, please manually specify the problem_type parameter during predictor init (You may specify problem_type as one of: ['binary', 'multiclass', 'regression'])
Fraction of data from classes w

We again demonstrate how to use the trained models to predict on the test data.

In [17]:
y_pred = predictor.predict(test_data_nolabel)
print("Predictions:  ", list(y_pred)[:5])
perf = predictor.evaluate_predictions(y_true=y_test, y_pred=y_pred, auxiliary_metrics=False)

Evaluation: accuracy on test data: 0.31746697420842945
Evaluations on test data:
{
    "accuracy": 0.31746697420842945
}


Predictions:   [' Exec-managerial', ' Exec-managerial', ' Craft-repair', ' Other-service', ' Craft-repair']


Use the following to view a summary of what happened during fit. Now this command will show details of the hyperparameter-tuning process for each type of model:

In [18]:
results = predictor.fit_summary()

*** Summary of fit() ***
Estimated performance of each model:
                  model  score_val  pred_time_val   fit_time  pred_time_val_marginal  fit_time_marginal  stack_level  can_infer  fit_order
0   WeightedEnsemble_L2   0.428571       0.163222  10.909559                0.000494           0.369986            2       True         14
1               XGBoost   0.397959       0.035605   4.510448                0.035605           4.510448            1       True         11
2              LightGBM   0.367347       0.009940   4.570944                0.009940           4.570944            1       True          5
3            LightGBMXT   0.357143       0.007098   3.951638                0.007098           3.951638            1       True          4
4              CatBoost   0.346939       0.008336   5.886357                0.008336           5.886357            1       True          8
5        NeuralNetTorch   0.346939       0.013867   5.135011                0.013867           5.135011 

In the above example, the predictive performance may be poor because we specified very little training to ensure quick runtimes. You can call `fit()` multiple times while modifying the above settings to better understand how these choices affect performance outcomes. For example: you can comment out the `train_data.head` command or increase `subsample_size` to train using a larger dataset, increase the `num_epochs` and `num_boost_round` hyperparameters, and increase the `time_limit` (which you should do for all code in these tutorials). To see more detailed output during the execution of `fit()`, you can also pass in the argument: `verbosity=3`.

## Model ensembling with stacking/bagging

Beyond hyperparameter-tuning with a correctly-specified evaluation metric, two other methods to boost predictive performance are bagging and stack-ensembling:

"AutoGluon-Tabular: Robust and Accurate AutoML for Structured Data": https://arxiv.org/abs/2003.06505

You’ll often see performance improve if you specify `num_bag_folds = 5-10, num_stack_levels = 1-3` in the call to `fit()`, but this will increase training times and memory/disk usage.

In [19]:
# last  argument is just for quick demo here, omit it in real applications
predictor = TabularPredictor(label=label, eval_metric=metric).fit(train_data,
    num_bag_folds=5, num_bag_sets=1, num_stack_levels=1,
    hyperparameters = {'NN': {'num_epochs': 2}, 'GBM': {'num_boost_round': 20}},  
)

No path specified. Models will be saved in: "AutogluonModels/ag-20230517_165833/"
Beginning AutoGluon training ...
AutoGluon will save models to "AutogluonModels/ag-20230517_165833/"
AutoGluon Version:  0.7.0
Python Version:     3.9.11
Operating System:   Linux
Platform Machine:   x86_64
Platform Version:   #1 SMP Tue Mar 7 15:41:52 UTC 2023
Train Data Rows:    500
Train Data Columns: 14
Label Column: occupation
Preprocessing data ...
AutoGluon infers your prediction problem is: 'multiclass' (because dtype of label-column == object).
	First 10 (of 15) unique label values:  [' Exec-managerial', ' Other-service', ' Craft-repair', ' Sales', ' Prof-specialty', ' Protective-serv', ' ?', ' Adm-clerical', ' Machine-op-inspct', ' Tech-support']
	If 'multiclass' is not the correct problem_type, please manually specify the problem_type parameter during predictor init (You may specify problem_type as one of: ['binary', 'multiclass', 'regression'])
Fraction of data from classes with at least 10 ex

	0.06s	 = Validation runtime
Fitting model: NeuralNetMXNet_BAG_L2 ...
	Fitting 5 child models (S1F1 - S1F5) | Fitting with ParallelLocalFoldFittingStrategy
		ray::_ray_fit() (pid=4565, ip=10.164.150.180)
ModuleNotFoundError: No module named 'mxnet'

During handling of the above exception, another exception occurred:

ray::_ray_fit() (pid=4565, ip=10.164.150.180)
  File "/home/stever7/.local/lib/python3.9/site-packages/autogluon/core/models/ensemble/fold_fitting_strategy.py", line 374, in _ray_fit
    fold_model.fit(X=X_fold, y=y_fold, X_val=X_val_fold, y_val=y_val_fold,
  File "/home/stever7/.local/lib/python3.9/site-packages/autogluon/core/models/abstract/abstract_model.py", line 703, in fit
    out = self._fit(**kwargs)
  File "/home/stever7/.local/lib/python3.9/site-packages/autogluon/tabular/models/tabular_nn/mxnet/tabular_nn_mxnet.py", line 135, in _fit
    try_import_mxnet()
  File "/home/stever7/.local/lib/python3.9/site-packages/autogluon/core/utils/try_import.py", line 49, in 

You should not provide `tuning_data` when stacking/bagging, and instead provide all your available data as `train_data` (which AutoGluon will split in more intellgent ways). `num_bag_sets` controls how many times the k-fold bagging process is repeated to further reduce variance (increasing this may further boost accuracy but will substantially increase training times, inference latency, and memory/disk usage). Rather than manually searching for good bagging/stacking values yourself, AutoGluon will automatically select good values for you if you specify `auto_stack` instead:

In [20]:
save_path = 'agModels-predictOccupation'  # folder where to store trained models

# last 2 arguments are for quick demo, omit them in real applications
predictor = TabularPredictor(label=label, eval_metric=metric, path=save_path).fit(
    train_data, 
    auto_stack=True,
    time_limit=30, 
    hyperparameters={'NN': {'num_epochs': 2}, 'GBM': {'num_boost_round': 20}} 
)

Stack configuration (auto_stack=True): num_stack_levels=0, num_bag_folds=5, num_bag_sets=20
Beginning AutoGluon training ... Time limit = 30s
AutoGluon will save models to "agModels-predictOccupation/"
AutoGluon Version:  0.7.0
Python Version:     3.9.11
Operating System:   Linux
Platform Machine:   x86_64
Platform Version:   #1 SMP Tue Mar 7 15:41:52 UTC 2023
Train Data Rows:    500
Train Data Columns: 14
Label Column: occupation
Preprocessing data ...
AutoGluon infers your prediction problem is: 'multiclass' (because dtype of label-column == object).
	First 10 (of 15) unique label values:  [' Exec-managerial', ' Other-service', ' Craft-repair', ' Sales', ' Prof-specialty', ' Protective-serv', ' ?', ' Adm-clerical', ' Machine-op-inspct', ' Tech-support']
	If 'multiclass' is not the correct problem_type, please manually specify the problem_type parameter during predictor init (You may specify problem_type as one of: ['binary', 'multiclass', 'regression'])
Fraction of data from classes 

Often stacking/bagging will produce superior accuracy than hyperparameter-tuning, but you may try combining both techniques (note: specifying `presets='best_quality'` in `fit()` simply sets `auto_stack=True`).

## Prediction options (inference)

Even if you’ve started a new Python session since last calling `fit()`, you can still load a previously trained predictor from disk:

```
# `predictor.path` is another way to get the relative path needed to later load predictor.
predictor = TabularPredictor.load(save_path)  
```

Above `save_path` is the same folder previously passed to `TabularPredictor`, in which all the trained models have been saved. You can train easily models on one machine and deploy them on another. Simply copy the `save_path` folder to the new machine and specify its new path in `TabularPredictor.load()`.

We can make a prediction on an individual example rather than a full dataset:

In [21]:
datapoint = test_data_nolabel.iloc[[0]]  # Note: .iloc[0] won't work because it returns pandas Series instead of DataFrame

print(datapoint)
predictor.predict(datapoint)

      age workclass  fnlwgt      education  education-num marital-status  \
5000   49   Private  259087   Some-college             10       Divorced   

        relationship    race      sex  capital-gain  capital-loss  \
5000   Not-in-family   White   Female             0             0   

      hours-per-week  native-country   class  
5000              40   United-States   <=50K  


5000     Exec-managerial
Name: occupation, dtype: object

To output predicted class probabilities instead of predicted classes, you can use:

In [22]:
predictor.predict_proba(datapoint)  # returns a DataFrame that shows which probability corresponds to which class

,?,Adm-clerical,Armed-Forces,Craft-repair,Exec-managerial,Farming-fishing,Handlers-cleaners,Machine-op-inspct,Other-service,Priv-house-serv,Prof-specialty,Protective-serv,Sales,Tech-support,Transport-moving
5000,0.036423,0.163815,0.0,0.13473,0.234789,0.014535,0.034552,0.048815,0.058644,0.0,0.088821,0.0,0.106273,0.01641,0.062193


By default, `predict()` and `predict_proba()` will utilize the model that AutoGluon thinks is most accurate, which is usually an ensemble of many individual models. Here’s how to see which model this is:

In [23]:
predictor.get_model_best()

'WeightedEnsemble_L2'

We can instead specify a particular model to use for predictions (e.g. to reduce inference latency). Note that a ‘model’ in AutoGluon may refer to for example a single Neural Network, a bagged ensemble of many Neural Network copies trained on different training/validation splits, a weighted ensemble that aggregates the predictions of many other models, or a stacker model that operates on predictions output by other models. This is akin to viewing a Random Forest as one ‘model’ when it is in fact an ensemble of many decision trees.

Before deciding which model to use, let’s evaluate all of the models AutoGluon has previously trained on our test data:

In [24]:
# predictor.leaderboard(test_data, silent=True)
predictor.leaderboard(test_data, silent=True).T

,0,1
model,LightGBM_BAG_L1,WeightedEnsemble_L2
score_test,0.283078,0.283078
score_val,0.329243,0.329243
pred_time_test,0.165294,0.168448
pred_time_val,0.240207,0.240737
fit_time,4.160288,4.164579
pred_time_test_marginal,0.165294,0.003155
pred_time_val_marginal,0.240207,0.00053
fit_time_marginal,4.160288,0.004291
stack_level,1,2


The leaderboard shows each model’s predictive performance on the test data (`score_test`) and validation data (`score_val`), as well as the time required to: produce predictions for the test data (`pred_time_val`), produce predictions on the validation data (`pred_time_val`), and train only this model (`fit_time`). Below, we show that a leaderboard can be produced without new data (just uses the data previously reserved for validation inside `fit`) and can display extra information about each model:

In [25]:
# predictor.leaderboard(extra_info=True, silent=True)
predictor.leaderboard(extra_info=True, silent=True).T

,0,1
model,LightGBM_BAG_L1,WeightedEnsemble_L2
score_val,0.329243,0.329243
pred_time_val,0.240207,0.240737
fit_time,4.160288,4.164579
pred_time_val_marginal,0.240207,0.00053
fit_time_marginal,4.160288,0.004291
stack_level,1,2
can_infer,True,True
fit_order,1,2
num_features,14,12


The expanded leaderboard shows properties like how many features are used by each model (`num_features`), which other models are ancestors whose predictions are required inputs for each model (`ancestors`), and how much memory each model and all its ancestors would occupy if simultaneously persisted (`memory_size_w_ancestors`). See the leaderboard documentation for full details:

https://auto.gluon.ai/0.1.0/api/autogluon.predictor.html#autogluon.tabular.TabularPredictor.leaderboard

Here’s how to specify a particular model to use for prediction instead of AutoGluon’s default model-choice:

In [26]:
i = 0  # index of model to use
model_to_use = predictor.get_model_names()[i]
model_pred = predictor.predict(datapoint, model=model_to_use)
print("Prediction from %s model: %s" % (model_to_use, model_pred.iloc[0]))

Prediction from LightGBM_BAG_L1 model:  Exec-managerial


We can easily access various information about the trained predictor or a particular model:

In [27]:
all_models = predictor.get_model_names()
model_to_use = all_models[i]
specific_model = predictor._trainer.load_model(model_to_use)

# Objects defined below are dicts of various information (not printed here as they are quite large):
model_info = specific_model.get_info()
predictor_information = predictor.info()

The `predictor` also remembers what metric predictions should be evaluated with, which can be done with ground truth labels as follows:

In [28]:
y_pred = predictor.predict(test_data_nolabel)
perf = predictor.evaluate_predictions(y_true=y_test, y_pred=y_pred, auxiliary_metrics=True)

Evaluation: accuracy on test data: 0.28307821346194173
Evaluations on test data:
{
    "accuracy": 0.28307821346194173,
    "balanced_accuracy": 0.17889516678234868,
    "mcc": 0.19219227485377302
}


However, you must be careful here as certain metrics require predicted probabilities rather than classes. Since the label columns remains in the `test_data` DataFrame, we can instead use the shorthand:

In [29]:
perf = predictor.evaluate(test_data)

Evaluation: accuracy on test data: 0.28307821346194173
Evaluations on test data:
{
    "accuracy": 0.28307821346194173,
    "balanced_accuracy": 0.17889516678234868,
    "mcc": 0.19219227485377302
}


which will correctly select between `predict()` or `predict_proba()` depending on the evaluation metric.

## Interpretability (feature importance)

To better understand our trained predictor, we can estimate the overall importance of each feature:

In [30]:
predictor.feature_importance(test_data)

Computing feature importance via permutation shuffling for 14 features using 4637 rows with 5 shuffle sets...
	14.87s	= Expected runtime (2.97s per shuffle set)
	4.69s	= Actual runtime (Completed 5 of 5 shuffle sets)


,importance,stddev,p_value,n,p99_high,p99_low
education-num,0.069355,0.002806,3.208365e-07,5,0.075133,0.063578
workclass,0.037265,0.002002,9.962103e-07,5,0.041388,0.033143
sex,0.036058,0.001175,1.352490e-07,5,0.038478,0.033638
hours-per-week,0.032219,0.005558,1.022038e-04,5,0.043664,0.020774
age,0.020617,0.004761,3.181865e-04,5,0.030419,0.010815
class,0.007850,0.001908,3.879995e-04,5,0.011779,0.003921
fnlwgt,0.002027,0.003740,1.461170e-01,5,0.009728,-0.005674
marital-status,0.001941,0.000341,1.097557e-04,5,0.002643,0.001239
relationship,0.001596,0.000947,9.831950e-03,5,0.003547,-0.000355
education,0.001035,0.000798,2.206371e-02,5,0.002679,-0.000608


Computed via permutation-shuffling, these feature importance scores quantify the drop in predictive performance (of the already trained predictor) when one column’s values are randomly shuffled across rows. 

https://explained.ai/rf-importance/

The top features in this list contribute most to AutoGluon’s accuracy (for predicting when/if a patient will be readmitted to the hospital). Features with non-positive importance score hardly contribute to the predictor’s accuracy, or may even be actively harmful to include in the data (consider removing these features from your data and calling `fit` again). These scores facilitate interpretability of the predictor’s global behavior (which features it relies on for all predictions) rather than local explanations that only rationalize one particular prediction.

https://christophm.github.io/interpretable-ml-book/taxonomy-of-interpretability-methods.html

## Accelerating inference

We describe multiple ways to reduce the time it takes for AutoGluon to produce predictions.

### Keeping models in memory

By default, AutoGluon loads models into memory one at a time and only when they are needed for prediction. This strategy is robust for large stacked/bagged ensembles, but leads to slower prediction times. If you plan to repeatedly make predictions (e.g. on new datapoints one at a time rather than one large test dataset), you can first specify that all models required for inference should be loaded into memory as follows:

In [31]:
predictor.persist_models()

num_test = 20
preds = np.array([''] * num_test, dtype='object')
for i in range(num_test):
    datapoint = test_data_nolabel.iloc[[i]]
    pred_numpy = predictor.predict(datapoint, as_pandas=False)
    preds[i] = pred_numpy[0]

perf = predictor.evaluate_predictions(y_test[:num_test], preds, auxiliary_metrics=True)
print("Predictions: ", preds)

predictor.unpersist_models()  # free memory by clearing models, future predict() calls will load models from disk

Persisting 2 models in memory. Models will require 0.0% of memory.
/tmp/ipykernel_62808/3174533836.py:10: FutureWarning: The behavior of `series[i:j]` with an integer-dtype index is deprecated. In a future version, this will be treated as *label-based* indexing, consistent with e.g. `series[i]` lookups. To retain the old behavior, use `series.iloc[i:j]`. To get the future behavior, use `series.loc[i:j]`.
  perf = predictor.evaluate_predictions(y_test[:num_test], preds, auxiliary_metrics=True)
Evaluation: accuracy on test data: 0.25
Evaluations on test data:
{
    "accuracy": 0.25,
    "balanced_accuracy": 0.3208333333333336,
    "mcc": 0.1275983604226418
}
Unpersisted 2 models: ['LightGBM_BAG_L1', 'WeightedEnsemble_L2']


Predictions:  [' Exec-managerial' ' Craft-repair' ' Craft-repair' ' Adm-clerical' ' ?'
 ' Exec-managerial' ' Exec-managerial' ' Sales' ' Exec-managerial'
 ' Adm-clerical' ' Other-service' ' Exec-managerial' ' Exec-managerial'
 ' Exec-managerial' ' Adm-clerical' ' ?' ' Craft-repair' ' Craft-repair'
 ' Exec-managerial' ' Craft-repair']


['LightGBM_BAG_L1', 'WeightedEnsemble_L2']

You can alternatively specify a particular model to persist via the models argument of `persist_models()`, or simply set `models='all'` to simultaneously load every single model that was trained during `fit`.

## Using smaller ensemble or faster model for prediction

Without having to retrain any models, one can construct alternative ensembles that aggregate individual models’ predictions with different weighting schemes. These ensembles become smaller (and hence faster for prediction) if they assign nonzero weight to less models. You can produce a wide variety of ensembles with different accuracy-speed tradeoffs like this:

In [32]:
additional_ensembles = predictor.fit_weighted_ensemble(expand_pareto_frontier=True)
print("Alternative ensembles you can use for prediction:", additional_ensembles)

predictor.leaderboard(only_pareto_frontier=True, silent=True)

Fitting model: WeightedEnsemble_L2Best ...
	0.3292	 = Validation score   (accuracy)
	0.0s	 = Training   runtime
	0.0s	 = Validation runtime


Alternative ensembles you can use for prediction: ['WeightedEnsemble_L2Best']


,model,score_val,pred_time_val,fit_time,pred_time_val_marginal,fit_time_marginal,stack_level,can_infer,fit_order
0,LightGBM_BAG_L1,0.329243,0.240207,4.160288,0.240207,4.160288,1,True,1


The resulting leaderboard will contain the most accurate model for a given inference-latency. You can select whichever model exhibits acceptable latency from the leaderboard and use it for prediction.

In [33]:
model_for_prediction = additional_ensembles[0]
predictions = predictor.predict(test_data, model=model_for_prediction)

# delete these extra models so they don't affect rest of tutorial
predictor.delete_models(models_to_delete=additional_ensembles, dry_run=False)  

Deleting model WeightedEnsemble_L2Best. All files under agModels-predictOccupation/models/WeightedEnsemble_L2Best/ will be removed.


### Collapsing bagged ensembles via `refit_full`

For an ensemble predictor trained with bagging (as done above), recall there ~10 bagged copies of each individual model trained on different train/validation folds. We can collapse this bag of ~10 models into a single model that’s fit to the full dataset, which can greatly reduce its memory/latency requirements (but may also reduce accuracy). Below we refit such a model for each original model but you can alternatively do this for just a particular model by specifying the model argument of `refit_full()`.

In [34]:
refit_model_map = predictor.refit_full()
print("Name of each refit-full model corresponding to a previous bagged ensemble:")
print(refit_model_map)
predictor.leaderboard(test_data, silent=True)

Refitting models via `predictor.refit_full` using all of the data (combined train and validation)...
	Models trained in this way will have the suffix "_FULL" and have NaN validation score.
	This process is not bound by time_limit, but should take less time than the original `predictor.fit` call.
	To learn more, refer to the `.refit_full` method docstring which explains how "_FULL" models differ from normal models.
Fitting 1 L1 models ...
Fitting model: LightGBM_BAG_L1_FULL ...
	0.39s	 = Training   runtime
Fitting model: WeightedEnsemble_L2_FULL | Skipping fit via cloning parent ...
	0.0s	 = Training   runtime
Updated best model to "WeightedEnsemble_L2_FULL" (Previously "WeightedEnsemble_L2"). AutoGluon will default to using "WeightedEnsemble_L2_FULL" for predict() and predict_proba().
Refit complete, total runtime = 0.76s


Name of each refit-full model corresponding to a previous bagged ensemble:
{'LightGBM_BAG_L1': 'LightGBM_BAG_L1_FULL', 'WeightedEnsemble_L2': 'WeightedEnsemble_L2_FULL'}


,model,score_test,score_val,pred_time_test,pred_time_val,fit_time,pred_time_test_marginal,pred_time_val_marginal,fit_time_marginal,stack_level,can_infer,fit_order
0,LightGBM_BAG_L1,0.283078,0.329243,0.167597,0.240207,4.160288,0.167597,0.240207,4.160288,1,True,1
1,WeightedEnsemble_L2,0.283078,0.329243,0.170756,0.240737,4.164579,0.003160,0.000530,0.004291,2,True,2
2,LightGBM_BAG_L1_FULL,0.273433,NaN,0.020983,NaN,0.389072,0.020983,NaN,0.389072,1,True,3
3,WeightedEnsemble_L2_FULL,0.273433,NaN,0.026119,NaN,0.393363,0.005136,NaN,0.004291,2,True,4


This adds the refit-full models to the leaderboard and we can opt to use any of them for prediction just like any other model. Note `pred_time_test` and `pred_time_val` list the time taken to produce predictions with each model (in seconds) on the test/validation data. Since the refit-full models were trained using all of the data, there is no internal validation score (`score_val`) available for them. You can also call `refit_full()` with non-bagged models to refit the same models to your full dataset (there won’t be memory/latency gains in this case but test accuracy may improve).

### Model distillation

While computationally-favorable, single individual models will usually have lower accuracy than weighted/stacked/bagged ensembles. Model Distillation offers one way to retain the computational benefits of a single model, while enjoying some of the accuracy-boost that comes with ensembling. 

"Fast, Accurate, and Simple Models for Tabular Data via Augmented Distillation": https://arxiv.org/abs/2006.14284

The idea is to train the individual model (which we can call the student) to mimic the predictions of the full stack ensemble (the teacher). Like `refit_full()`, the `distill()` function will produce additional models we can opt to use for prediction.

In [35]:
student_models = predictor.distill(time_limit=30)  # specify much longer time limit in real applications
print(student_models)

preds_student = predictor.predict(test_data_nolabel, model=student_models[0])
print(f"predictions from {student_models[0]}:", list(preds_student)[:5])
predictor.leaderboard(test_data)

Distilling with teacher='WeightedEnsemble_L2_FULL', teacher_preds=soft, augment_method=spunge ...
SPUNGE: Augmenting training data with 1955 synthetic samples for distillation...
Distilling with each of these student models: ['LightGBM_DSTL', 'NeuralNetMXNet_DSTL', 'RandomForestMSE_DSTL', 'CatBoost_DSTL', 'NeuralNetTorch_DSTL']
Fitting 5 L1 models ...
Fitting model: LightGBM_DSTL ... Training model for up to 30.0s of the 29.99s of remaining time.
	Note: model has different eval_metric than default.
	-1.8088	 = Validation score   (-soft_log_loss)
	16.61s	 = Training   runtime
	0.02s	 = Validation runtime
Fitting model: NeuralNetMXNet_DSTL ... Training model for up to 12.99s of the 12.98s of remaining time.
		Unable to import dependency mxnet. A quick tip is to install via `pip install mxnet --upgrade`, or `pip install mxnet_cu101 --upgrade`
Fitting model: RandomForestMSE_DSTL ... Training model for up to 12.94s of the 12.94s of remaining time.
	Note: model has different eval_metric than

['LightGBM_DSTL', 'RandomForestMSE_DSTL', 'CatBoost_DSTL', 'WeightedEnsemble_L2_DSTL']
predictions from LightGBM_DSTL: [' Exec-managerial', ' Exec-managerial', ' Craft-repair', ' Other-service', ' Craft-repair']
                      model  score_test  score_val  pred_time_test  pred_time_val   fit_time  pred_time_test_marginal  pred_time_val_marginal  fit_time_marginal  stack_level  can_infer  fit_order
0             CatBoost_DSTL    0.317048   0.295918        0.026672       0.009593  11.143906                 0.026672                0.009593          11.143906            1       True          7
1      RandomForestMSE_DSTL    0.298805   0.326531        0.562040       0.073656   0.822182                 0.562040                0.073656           0.822182            1       True          6
2           LightGBM_BAG_L1    0.283078   0.329243        0.470379       0.240207   4.160288                 0.470379                0.240207           4.160288            1       True          1
3   

,model,score_test,score_val,pred_time_test,pred_time_val,fit_time,pred_time_test_marginal,pred_time_val_marginal,fit_time_marginal,stack_level,can_infer,fit_order
0,CatBoost_DSTL,0.317048,0.295918,0.026672,0.009593,11.143906,0.026672,0.009593,11.143906,1,True,7
1,RandomForestMSE_DSTL,0.298805,0.326531,0.562040,0.073656,0.822182,0.562040,0.073656,0.822182,1,True,6
2,LightGBM_BAG_L1,0.283078,0.329243,0.470379,0.240207,4.160288,0.470379,0.240207,4.160288,1,True,1
3,WeightedEnsemble_L2,0.283078,0.329243,0.474625,0.240737,4.164579,0.004246,0.000530,0.004291,2,True,2
4,LightGBM_DSTL,0.282239,0.438776,0.146689,0.017938,16.606914,0.146689,0.017938,16.606914,1,True,5
5,WeightedEnsemble_L2_DSTL,0.282239,0.438776,0.151303,0.018474,16.724060,0.004614,0.000536,0.117146,2,True,8
6,LightGBM_BAG_L1_FULL,0.273433,NaN,0.024070,NaN,0.389072,0.024070,NaN,0.389072,1,True,3
7,WeightedEnsemble_L2_FULL,0.273433,NaN,0.030887,NaN,0.393363,0.006818,NaN,0.004291,2,True,4


### Faster presets or hyperparameters

Instead of trying to speed up a cumbersome trained model at prediction time, if you know inference latency or memory will be an issue at the outset, then you can adjust the training process accordingly to ensure `fit()` does not produce unwieldy models.

One option is to specify more lightweight presets:

In [36]:
presets = ['good_quality_faster_inference_only_refit', 'optimize_for_deployment']
predictor_light = TabularPredictor(label=label, eval_metric=metric).fit(train_data, presets=presets, time_limit=30)

No path specified. Models will be saved in: "AutogluonModels/ag-20230517_170031/"
Preset alias specified: 'good_quality_faster_inference_only_refit' maps to 'good_quality'.
Presets specified: ['good_quality_faster_inference_only_refit', 'optimize_for_deployment']
Stack configuration (auto_stack=True): num_stack_levels=0, num_bag_folds=5, num_bag_sets=20
Beginning AutoGluon training ... Time limit = 30s
AutoGluon will save models to "AutogluonModels/ag-20230517_170031/"
AutoGluon Version:  0.7.0
Python Version:     3.9.11
Operating System:   Linux
Platform Machine:   x86_64
Platform Version:   #1 SMP Tue Mar 7 15:41:52 UTC 2023
Train Data Rows:    500
Train Data Columns: 14
Label Column: occupation
Preprocessing data ...
AutoGluon infers your prediction problem is: 'multiclass' (because dtype of label-column == object).
	First 10 (of 15) unique label values:  [' Exec-managerial', ' Other-service', ' Craft-repair', ' Sales', ' Prof-specialty', ' Protective-serv', ' ?', ' Adm-clerical', '

Another option is to specify more lightweight hyperparameters:

In [37]:
predictor_light = TabularPredictor(label=label, eval_metric=metric).fit(train_data, hyperparameters='very_light', time_limit=30)

No path specified. Models will be saved in: "AutogluonModels/ag-20230517_170108/"
Beginning AutoGluon training ... Time limit = 30s
AutoGluon will save models to "AutogluonModels/ag-20230517_170108/"
AutoGluon Version:  0.7.0
Python Version:     3.9.11
Operating System:   Linux
Platform Machine:   x86_64
Platform Version:   #1 SMP Tue Mar 7 15:41:52 UTC 2023
Train Data Rows:    500
Train Data Columns: 14
Label Column: occupation
Preprocessing data ...
AutoGluon infers your prediction problem is: 'multiclass' (because dtype of label-column == object).
	First 10 (of 15) unique label values:  [' Exec-managerial', ' Other-service', ' Craft-repair', ' Sales', ' Prof-specialty', ' Protective-serv', ' ?', ' Adm-clerical', ' Machine-op-inspct', ' Tech-support']
	If 'multiclass' is not the correct problem_type, please manually specify the problem_type parameter during predictor init (You may specify problem_type as one of: ['binary', 'multiclass', 'regression'])
Fraction of data from classes wi

Here you can set hyperparameters to either 'light', 'very_light', or 'toy' to obtain progressively smaller (but less accurate) models and predictors. Advanced users may instead try manually specifying particular models’ hyperparameters in order to make them faster/smaller.

Finally, you may also exclude specific unwieldy models from being trained at all. Below we exclude models that tend to be slower (K Nearest Neighbors, Neural Network, models with custom larger-than-default hyperparameters):

In [38]:
excluded_model_types = ['KNN', 'NN', 'custom']

predictor_light = TabularPredictor(label=label, eval_metric=metric).fit(
    train_data, 
    excluded_model_types=excluded_model_types, 
    time_limit=30
)

No path specified. Models will be saved in: "AutogluonModels/ag-20230517_170139/"
Beginning AutoGluon training ... Time limit = 30s
AutoGluon will save models to "AutogluonModels/ag-20230517_170139/"
AutoGluon Version:  0.7.0
Python Version:     3.9.11
Operating System:   Linux
Platform Machine:   x86_64
Platform Version:   #1 SMP Tue Mar 7 15:41:52 UTC 2023
Train Data Rows:    500
Train Data Columns: 14
Label Column: occupation
Preprocessing data ...
AutoGluon infers your prediction problem is: 'multiclass' (because dtype of label-column == object).
	First 10 (of 15) unique label values:  [' Exec-managerial', ' Other-service', ' Craft-repair', ' Sales', ' Prof-specialty', ' Protective-serv', ' ?', ' Adm-clerical', ' Machine-op-inspct', ' Tech-support']
	If 'multiclass' is not the correct problem_type, please manually specify the problem_type parameter during predictor init (You may specify problem_type as one of: ['binary', 'multiclass', 'regression'])
Fraction of data from classes wi

## If you encounter memory issues

To reduce memory usage during training, you may try each of the following strategies individually or combinations of them (these may harm accuracy):

- In `fit()`, set `num_bag_sets=1` (can also try values greater than 1 to harm accuracy less)

- In `fit()`, set `excluded_model_types=['KNN', 'XT', 'RF']` (or some subset of these models)

- Try different `presets` in `fit()`

- In `fit()`, set `hyperparameters='light'` or `hyperparameters='very_light'`

- Text fields in your table require substantial memory for N-gram featurization. To mitigate this in `fit()`, you can either:
    + (1) add `'ignore_text'` to your `presets` list (to ignore text features), or 
    + (2) specify the argument:

```
from sklearn.feature_extraction.text import CountVectorizer
from autogluon.features.generators import AutoMLPipelineFeatureGenerator

feature_generator = AutoMLPipelineFeatureGenerator(
    vectorizer=CountVectorizer(
        min_df=30, 
        ngram_range=(1, 3), 
        max_features=MAX_NGRAM, 
        dtype=np.uint8
    )
)
```

where `MAX_NGRAM=1000` say (try various values under 10000 to reduce the number of N-gram features used to represent each text field).

In addition to reducing memory usage, many of the above strategies can also be used to reduce training times.

To reduce memory usage during inference:

- If trying to produce predictions for a large test dataset, break the test data into smaller chunks as demonstrated in FAQ.
    + https://auto.gluon.ai/0.1.0/tutorials/tabular_prediction/tabular-faq.html#sec-faq

- If models have been previously persisted in memory but inference-speed is not a major concern, call `predictor.unpersist_models()`

- If models have been previously persisted in memory, bagging was used in `fit()`, and inference-speed is a concern: call `predictor.refit_full()` and use one of the refit-full models for prediction (ensure this is the only model persisted in memory)

## If you encounter disk space issues

To reduce disk usage, you may try each of the following strategies individually or combinations of them:

- Make sure to delete all `predictor.path` folders from previous `fit()` runs
    + These can eat up your free space if you call `fit()` many times
    + If you didn’t specify path, AutoGluon still automatically saved its models to a folder called: `“AutogluonModels/ag-[TIMESTAMP]”`, where TIMESTAMP records when `fit()` was called, so make sure to also delete these folders if you run low on free space

- Call `predictor.save_space()` to delete auxiliary files produced during `fit()`

- Call `predictor.delete_models(models_to_keep='best', dry_run=False)` if you only intend to use this predictor for inference going forward (will delete files required for non-prediction-related functionality like `fit_summary`)

- In `fit()`, you can add `'optimize_for_deployment'` to the presets list, which will automatically invoke the previous two strategies after training

Most of the above strategies to reduce memory usage will also reduce disk usage (but may harm accuracy).

## References

The following paper describes how AutoGluon internally operates on tabular data:

Erickson et al. "AutoGluon-Tabular: Robust and Accurate AutoML for Structured Data". Arxiv, 2020: https://arxiv.org/abs/2003.06505